In [ ]:
# Ce projet fonctionne avec les versions des package suivants:
# pandas 2.2.2
# matplotlib 3.9.0

In [ ]:
# Nous allons réaliser une analyse afin d'obtimiser les tourner des personne de l'entretien
# Le contexte
# Un markdown pour ce que j'ai obtenu

In [195]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [197]:
data= pd.read_csv("datas/p2-arbres-fr.csv", sep=";") #chargement du fichier CSV dans la variables data

In [198]:
# On va realiser une annalyse naïve du jeu de données pour nous rendre compte des éléments à notre disposition
# et choisir la meilleurs approche possible pour le challenge
print(data.head(10)) #Affiche les 10 première ligne du fichier pour vérifier que ce dernier est bien chargé

      id type_emplacement domanialite   arrondissement complement_addresse  \
0  99874            Arbre      Jardin   PARIS 7E ARRDT                 NaN   
1  99875            Arbre      Jardin   PARIS 7E ARRDT                 NaN   
2  99876            Arbre      Jardin   PARIS 7E ARRDT                 NaN   
3  99877            Arbre      Jardin   PARIS 7E ARRDT                 NaN   
4  99878            Arbre      Jardin  PARIS 17E ARRDT                 NaN   
5  99879            Arbre      Jardin  PARIS 17E ARRDT                 NaN   
6  99880            Arbre      Jardin  PARIS 17E ARRDT                 NaN   
7  99881            Arbre      Jardin  PARIS 16E ARRDT                 NaN   
8  99882            Arbre      Jardin  PARIS 16E ARRDT                 NaN   
9  99883            Arbre      Jardin  PARIS 16E ARRDT                 NaN   

   numero                                               lieu id_emplacement  \
0     NaN          MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E  

In [201]:
# dimension du dataset
dimension = data.shape
print(f"Le nombre de ligne du dataFrame {dimension[0]}")
print(f"Le nombre de colonne du dataFrame {dimension[1]}")

Le nombre de ligne du dataFrame 200137
Le nombre de colonne du dataFrame 18


In [203]:
# on identifie les colonnes ayant des données non null pour nettoyer notre jeu de données
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200137 entries, 0 to 200136
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   200137 non-null  int64  
 1   type_emplacement     200137 non-null  object 
 2   domanialite          200136 non-null  object 
 3   arrondissement       200137 non-null  object 
 4   complement_addresse  30902 non-null   object 
 5   numero               0 non-null       float64
 6   lieu                 200137 non-null  object 
 7   id_emplacement       200137 non-null  object 
 8   libelle_francais     198640 non-null  object 
 9   genre                200121 non-null  object 
 10  espece               198385 non-null  object 
 11  variete              36777 non-null   object 
 12  circonference_cm     200137 non-null  int64  
 13  hauteur_m            200137 non-null  int64  
 14  stade_developpement  132932 non-null  object 
 15  remarquable      

In [205]:
# on transforme ces résultat en pourcentage pour une meilleur lecture
non_null_percentage = (data.isnull().sum()/len(data))*100
print(f"\nNombre de données null en % :\n{non_null_percentage.round(2)}")


Nombre de données null en % :
id                       0.00
type_emplacement         0.00
domanialite              0.00
arrondissement           0.00
complement_addresse     84.56
numero                 100.00
lieu                     0.00
id_emplacement           0.00
libelle_francais         0.75
genre                    0.01
espece                   0.88
variete                 81.62
circonference_cm         0.00
hauteur_m                0.00
stade_developpement     33.58
remarquable             31.53
geo_point_2d_a           0.00
geo_point_2d_b           0.00
dtype: float64


In [207]:
# La colonne "numero" ayant que des valeur null, nous pouvons nous en passer
data = data.drop(columns =['numero'])
data.describe()

,id,circonference_cm,hauteur_m,remarquable,geo_point_2d_a,geo_point_2d_b
count,2.001370e+05,200137.000000,200137.000000,137039.000000,200137.000000,200137.000000
mean,3.872027e+05,83.380479,13.110509,0.001343,48.854491,2.348208
std,5.456032e+05,673.190213,1971.217387,0.036618,0.030234,0.051220
min,9.987400e+04,0.000000,0.000000,0.000000,48.742290,2.210241
25%,1.559270e+05,30.000000,5.000000,0.000000,48.835021,2.307530
50%,2.210780e+05,70.000000,8.000000,0.000000,48.854162,2.351095
75%,2.741020e+05,115.000000,12.000000,0.000000,48.876447,2.386838
max,2.024745e+06,250255.000000,881818.000000,1.000000,48.911485,2.469759


In [209]:
# on remarque un arbre de 881 kms et un autre arbre de circonference 2500m sont des valeurs abérantes
# nous avons aussi des arbres ayant une hauteur de 0 et une circonférence de 0
# nous allons donc travailler notre dataset pour le rendre plus cohérent.

# commencons par la hauteur et affichons les 10 valeurs les plus grandes pour determiner lesquels des valeur sont "Impossible", "abérantes" ou "athypique"


In [211]:
# En 2021 un des arbres le plus haut à Paris faisait 30m
# https://agriculture.gouv.fr/lun-des-plus-grands-arbres-de-paris-veille-sur-le-78-rue-de-varenne
# on peut retrouver une data d'un arbre de 45m en 2006
# https://public.opendatasoft.com/explore/dataset/arbresremarquablesparis2011/table/?flg=fr-fr&sort=arbres_hauteurenm

max_height=50

num_tall_trees = (data['hauteur_m']>max_height).sum()
print(f"Il y a {num_tall_trees} arbres ayant une hauteur de {max_height} m soit {((num_tall_trees/dimension[0])*100).round(2)} % du total des arbres")

Il y a 445 arbres ayant une hauteur de 50 m soit 0.22 % du total des arbres


In [213]:
# on retire donc dans un premier temps la hauteur abérante en considérant un hauteur max de 50 m
data = data[data['hauteur_m'] <= max_height]
data.describe()

,id,circonference_cm,hauteur_m,remarquable,geo_point_2d_a,geo_point_2d_b
count,1.996920e+05,199692.000000,199692.000000,136599.000000,199692.000000,199692.000000
mean,3.853691e+05,83.393766,8.333484,0.001340,48.854528,2.348208
std,5.432361e+05,673.615295,6.206695,0.036577,0.030233,0.051250
min,9.987400e+04,0.000000,0.000000,0.000000,48.742290,2.210241
25%,1.557658e+05,30.000000,5.000000,0.000000,48.835063,2.307484
50%,2.209215e+05,70.000000,8.000000,0.000000,48.854208,2.351170
75%,2.738762e+05,115.000000,12.000000,0.000000,48.876488,2.386878
max,2.024745e+06,250255.000000,50.000000,1.000000,48.911485,2.469759


In [215]:
# Pour finir le travail sur la hauteur, on veut savoir combien d'arbre ont une hauteur de zero

hauteur_zero_counts = (data['hauteur_m'] == 0).sum()
print(f" il y a {hauteur_zero_counts} arbres de hauteur 0")

 il y a 39219 arbres de hauteur 0


In [217]:
#On va remplacer la valeur des arbres ayant une hauteur null par la  moyenne de la hauteur des arbres ayant une hauteur non null

# on selectionne les arbres ayant une hauteur non null
non_hauteur_null_list = data[data['hauteur_m'] != 0]

# on calcul la hauteur moyenne
mean_hauteur = non_hauteur_null_list['hauteur_m'].mean()
print(f"Hauteur moyenne calculée : {mean_hauteur.round(3)} m")

# On remplace la valeur de la hauteur par la moyenne caclulée
data['hauteur_m']= data['hauteur_m'].replace(0, mean_hauteur)
data.describe()

Hauteur moyenne calculée : 10.37 m


,id,circonference_cm,hauteur_m,remarquable,geo_point_2d_a,geo_point_2d_b
count,1.996920e+05,199692.000000,199692.000000,136599.000000,199692.000000,199692.000000
mean,3.853691e+05,83.393766,10.370156,0.001340,48.854528,2.348208
std,5.432361e+05,673.615295,4.642241,0.036577,0.030233,0.051250
min,9.987400e+04,0.000000,1.000000,0.000000,48.742290,2.210241
25%,1.557658e+05,30.000000,7.000000,0.000000,48.835063,2.307484
50%,2.209215e+05,70.000000,10.000000,0.000000,48.854208,2.351170
75%,2.738762e+05,115.000000,12.000000,0.000000,48.876488,2.386878
max,2.024745e+06,250255.000000,50.000000,1.000000,48.911485,2.469759


In [219]:
# En 2006 la circonférence max des arbres à Paris était de 740 cm
# https://public.opendatasoft.com/explore/dataset/arbresremarquablesparis2011/table/?flg=fr-fr&sort=arbres_hauteurenm
# https://www.unjourdeplusaparis.com/paris-vert/arbres-remarquables-paris

# Les arbres au dessus seront donc considéré comme des valeurs abérantes.

max_circonference = 740
sorted_circonference = (data['circonference_cm']>max_circonference).sum()
numb_circonference_null = (data['circonference_cm']==0).sum()

print(f"Il y a {sorted_circonference} arbres ayant une circonference de plus de {max_circonference/100} m soit {((sorted_circonference/dimension[0])*100).round(2)} % du total des arbres")
print(f"et {numb_circonference_null} arbres ayant une circonference null soit {((numb_circonference_null/dimension[0])*100).round(2)} % du total des arbres")

Il y a 79 arbres ayant une circonference de plus de 7.4 m soit 0.04 % du total des arbres
et 25867 arbres ayant une circonference null soit 12.92 % du total des arbres


In [221]:
# On peut raisonnablement supprimer du dataset les arbres ayant une circonférence supérieurs à max_circonference
data = data[data['circonference_cm'] <= max_circonference]

# on selectionne les arbres ayant une circonference non null
non_circonference_null_list = data[data['circonference_cm'] != 0]

# on calcul la circonference moyenne
mean_circonference = non_circonference_null_list['circonference_cm'].mean()
print(f"Circonference moyenne calculée : {mean_circonference.round(3)} cm")

# On remplace la valeur de la circonference par la moyenne caclulée
data['circonference_cm']= data['circonference_cm'].replace(0, mean_hauteur)

Circonference moyenne calculée : 91.559 cm


In [223]:
# Calcul des caractéristiques principales pour les variables quantitatives
quantitative_columns = ['circonference_cm', 'hauteur_m', 'geo_point_2d_a', 'geo_point_2d_b']

# Describe pour obtenir les statistiques de base
stats_summary = data[quantitative_columns].describe().transpose()

# Affiche le résumé des statistiques
print(stats_summary)

                     count       mean        std        min        25%  \
circonference_cm  199613.0  81.038469  61.294525   1.000000  30.000000   
hauteur_m         199613.0  10.368737   4.641351   1.000000   7.000000   
geo_point_2d_a    199613.0  48.854528   0.030235  48.742290  48.835062   
geo_point_2d_b    199613.0   2.348211   0.051253   2.210241   2.307485   

                        50%         75%         max  
circonference_cm  70.000000  115.000000  710.000000  
hauteur_m         10.000000   12.000000   50.000000  
geo_point_2d_a    48.854207   48.876491   48.911485  
geo_point_2d_b     2.351176    2.386882    2.469759  


In [225]:
unique_stade_developpement = data['stade_developpement'].unique()
print(f'List des valeurs possible dans la colonne stade de développement {unique_stade_developpement}')

total_nan = data['stade_developpement'].isnull().sum()
print(f'total nan {total_nan}')


stade_developpement_counts = data['stade_developpement'].value_counts()
print(f'Nombre d\individus pour chaque cas dans {stade_developpement_counts}')


List des valeurs possible dans la colonne stade de développement [nan 'A' 'J' 'M' 'JA']
total nan 67197
Nombre d\individus pour chaque cas dans stade_developpement
A     64278
JA    35160
J     26878
M      6100
Name: count, dtype: int64


In [227]:
# Pour nous aider dans notre approche, on s'assure de n'avoir que des arbres dans notre dataset
nombre_valeur_type_emplacement = data['type_emplacement']
print(f'La liste contient {nombre_valeur_type_emplacement.nunique()} valeur')
print(f'\n: {nombre_valeur_type_emplacement.unique()}')

La liste contient 1 valeur

: ['Arbre']


In [ ]:
# Maintenant que l'on a 100% des valeurs hauteur_m et circonference_m dans notre dataSet
# On veut compléter la colonne stade de développement pour chaque arbres de notre dataSet
# Une approche simple consiste a determnier le ratio H/D pour chaque individus puis on définit le ratio moyen pour chaque stade_developpemt
data['ratio_H_D'] = ((data['hauteur_m']*100)/(data['circonference_cm'])).round(2)
data_filtered = data.dropna(subset=['stade_developpement'])
stats = data_filtered.groupby('stade_developpement').agg({
    'hauteur_m': ['max', 'min', 'mean'],
   'circonference_cm': ['max', 'min', 'mean'],
  'ratio_H_D': ['max', 'min', 'mean']
}).reset_index()
print(stats)

In [243]:
# on constate une coherence. Plus notre arbre est jeune plus le ratio H/D est grand.
# Cela est due à la pousse des arbres qui dans un premier temps grandissent plus qu'il ne grossisse
# On extrait les ratio poyen de chaque stade de developpement

mean_ratios = stats.set_index('stade_developpement')['ratio_H_D']['mean']
print(mean_ratios)

stade_developpement
A     11.573626
J     17.317323
JA    14.036608
M      9.719728
Name: mean, dtype: float64


In [255]:
def impute_stade_developppement(row,mean_ratios):
    if pd.isna(row['stade_developpement']):
        if row['ratio_H_D']<=mean_ratios['M']:
            return 'M'
        elif row['ratio_H_D']<=mean_ratios['A']:
            return 'A'
        elif row['ratio_H_D']<= mean_ratios['JA']:
            return 'JA'
        else:
            return 'J'
    else:
        return row['stade_developpement']

In [261]:
data_copy = data.copy()
data_copy['stade_developpement'] = data.apply(impute_stade_developppement, axis =1, mean_ratios = mean_ratios)
data_copy

,id,type_emplacement,domanialite,arrondissement,complement_addresse,lieu,id_emplacement,libelle_francais,genre,espece,variete,circonference_cm,hauteur_m,stade_developpement,remarquable,geo_point_2d_a,geo_point_2d_b,ratio_H_D
0,99874,Arbre,Jardin,PARIS 7E ARRDT,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,19,Marronnier,Aesculus,hippocastanum,NaN,20.000000,5.000000,J,0.0,48.857620,2.320962,25.00
1,99875,Arbre,Jardin,PARIS 7E ARRDT,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,20,If,Taxus,baccata,NaN,65.000000,8.000000,A,NaN,48.857656,2.321031,12.31
2,99876,Arbre,Jardin,PARIS 7E ARRDT,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,21,If,Taxus,baccata,NaN,90.000000,10.000000,A,NaN,48.857705,2.321061,11.11
3,99877,Arbre,Jardin,PARIS 7E ARRDT,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,22,Erable,Acer,negundo,NaN,60.000000,8.000000,A,NaN,48.857722,2.321006,13.33
4,99878,Arbre,Jardin,PARIS 17E ARRDT,NaN,PARC CLICHY-BATIGNOLLES-MARTIN LUTHER KING,000G0037,Arbre à miel,Tetradium,daniellii,NaN,38.000000,10.370156,J,NaN,48.890435,2.315289,27.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200132,2024741,Arbre,Jardin,BOIS DE VINCENNES,NaN,ARBORETUM DE L ECOLE DU BREUIL - ROUTE DE LA F...,720170151,NaN,Castanea,fargesii,NaN,10.370156,10.370156,J,0.0,48.821957,2.456264,100.00
200133,2024742,Arbre,Jardin,BOIS DE VINCENNES,NaN,ARBORETUM DE L ECOLE DU BREUIL - ROUTE DE LA F...,720170152,Arbre aux mouchoirs,Davidia,involucrata,NaN,10.370156,10.370156,J,0.0,48.823254,2.457935,100.00
200134,2024743,Arbre,Jardin,BOIS DE VINCENNES,NaN,ARBORETUM DE L ECOLE DU BREUIL - ROUTE DE LA F...,720170153,Peuplier,Populus,simonii,NaN,10.370156,10.370156,J,0.0,48.822907,2.457970,100.00
200135,2024744,Arbre,Jardin,BOIS DE VINCENNES,NaN,ARBORETUM DE L ECOLE DU BREUIL - ROUTE DE LA F...,720170154,Chêne,Quercus,n. sp.,NaN,10.370156,10.370156,J,0.0,48.822522,2.455956,100.00


In [ ]:
# On peut classer les arbres comme suit:
#https://bauchery.fr/blog/72_guide-des-tailles-et-qualite-des-plants.html#:~:text=%2D%20Un%20baliveau%20est%20un%20arbre,2%20m%20et%202.50%20m.&text=Ce%20rapport%20est%20variable%20suivant,plus%20grand%20qu'un%20noyer.
# Jeune ou baliveau H < 2.5m et circonference ~ 6cm
# Jeune Adulte 2.5m < H < 4m et circonference < 15cm

data_min=data[(data['circonference_cm']<=16) & (data['hauteur_m']< 4) & (data['stade_developpement'].isnull())]
data_min.describe()

In [ ]:
data['age']=(data['circonference_cm']/2.5).round(1)
data_ja = data[data['stade_developpement']=='A']
data_ja['age'].min()

In [ ]:
# Autre façon d'obtenir les statistiques avec les methodes
stats_dict={}

for column in quantitative_columns:
    stats_dict[column]={
        'Nb de valeur (count)':data[column].count(),
        'Moyenne (mean)':data[column].mean(),
        'std (ecart type)':data[column].std(),
        'mini':data[column].min(),
        '25%':data[column].quantile(0.25),
        '50% mediane':data[column].median(),
        '75%':data[column].quantile(0.75),
        'maxi':data[column].max()
    }

#conversion du dictionnaire en Dataframe pour un meilleur affichage
stats_df=pd.DataFrame(stats_dict).transpose()
print(stats_df)

In [ ]:
def create_graphics(data):
    plt.figure(figsize=(12,10))

    # Box plot pour circonference_cm
    plt.subplot(2,2,1)
    plt.boxplot(data['circonference_cm'])
    plt.title('Box Plot de la Circonférence des Arbres')
    plt.xlabel('Circonférence (cm)')
    plt.xticks([])
    
    # Histogramme avec courbe de densité pour circonference_cm
    plt.subplot(2,2,2)
    plt.hist(data['circonference_cm'], bins=range(0,700,10))
    plt.title('Distribution de la Circonférence des Arbres')
    plt.xlabel('Circonférence (cm)')
    plt.ylabel('Fréquence')

    # Box plot pour la hauteur_m
    plt.subplot(2,2,3)
    plt.boxplot(data['hauteur_m'])
    plt.title('Box Plot de la Hauteur des Arbres')
    plt.xlabel('Hauteur (m)')
    plt.xticks([])

    # Histogramme avec courbe de densité pour hauteur_m
    plt.subplot(2,2,4)
    plt.hist(data['hauteur_m'], bins =range(0,50,1), color='green')
    plt.title('Distribution de la Hauteur des Arbres')
    plt.xlabel('Hauteur (m)')
    plt.ylabel('Fréquence')

    plt.figure(figsize=(10,6))
    plt.scatter(data['geo_point_2d_a'],data['geo_point_2d_b'], s=data['hauteur_m'], alpha=0.5)
    plt.title('Distribution Géographique des Arbres avec la Taille des Points Représentant la Hauteur')
    plt.xlabel('Latitude (geo_point_2d_a)')
    plt.ylabel('Longitude (geo_point_2d_b)')
    plt.colorbar(label='Hauteur (m)')

create_graphics(data)

In [ ]:
def plot_qualitative(data,column, title,ylabel):
    plt.figure(figsize=(10,6))
    data[column].value_counts().plot(kind='bar')
    plt.title(title)
    plt.xlabel(column)
    plt.ylabel(ylabel)

In [ ]:
plot_qualitative(data,'arrondissement','Distribution des arbres par arrondissements','Nombre')

In [ ]:
variableX='type_emplacement'
variableY='espece'
grouped_data = data.groupby([variableX, variableY]).size().reset_index(name='count')

In [ ]:
pivot_data = grouped_data.pivot(index=variableX, columns=variableY, values='count').fillna(0)

# Créer un graphique en barres empilé
pivot_data.plot(kind='bar', stacked=True, figsize=(14, 8), legend=False)

plt.title(f'Nombre de {variableY} par {variableX}')
plt.xlabel(variableX)
plt.ylabel(f'Nombre {variableY}')
# plt.legend(title=variableY)

In [ ]:
top_10_cirocnference = data.nlargest(10,'hauteur_m')
top_10_cirocnference

In [ ]:
print(data.columns)

In [ ]:
# Nettoyage des données

# Detecter les outliers

# Fonction pour détecter les outliers en dehors des moustaches pour 'circonference_cm et hauteur_m
def detect_outliers_(data, column):
    Q1=data[column].quantile(0.25)
    Q3=data[column].quantile(0.75)
    IQR = Q3-Q1
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 +1.5 * IQR
    outliers = (data['column']<lower_limit | data['column']>upper_limit)
    return outliers

# Detect les oultiers pour la circonference et la hauteur
outliers_circonference = detect_outliers(data, 'circonference_cm')
outliers_hauteur = detect_outliers(data, 'hauteur_m')
outliers_circonference = outliers_circonference.astype(bool)
outliers_hauteur = outliers_hauteur.astype(bool)

# choix des colonnes à afficher
columnsToDisplay = ['circonference_cm','hauteur_m','libelle_francais','geo_point_2d_a','geo_point_2d_b']

# list des outliers du dataset
outliers_combined = outliers_circonference | outliers_hauteur
print('DataFrame sans les outliers')

# masque pour récupérer les valeur du dataset hors outlier
clean_data = data[~outliers_combined]
print(clean_data[columnsToDisplay])


In [ ]:
# On affiche de nouveau les graphiques avec le dataFrame nettoyé

create_graphics(clean_data)